<a href="https://colab.research.google.com/github/JaimeLopezGarrido/Trabajos_Labo1/blob/main/src/workflows/Gerencial_Final_20_E4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WorkFlow Experimental Jaime



### 9.5 Conexion con drive

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Drive already mounted at /content/.drive; to attempt to forcibly remount, call drive.mount("/content/.drive", force_remount=True).


In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

## 9.7  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Nov 29 12:29:22 PM 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660355,35.3,1454505,77.7,1454505,77.7
Vcells,1226516,9.4,8388608,64.0,1975136,15.1


In [ ]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings




#### Parametros


In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- 100007

PARAM$experimento <- 18
PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [ ]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 9.7.1   Preprocesamiento del dataset

#### 9.7.1.1  DT dataset gerencial

In [ ]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

#### 9.7.1.2  CA  Catastrophe Analysis
Opto por metodo del vago, no hacer nada.

#### 9.7.1.3  DR  Data Drifting
Se intenta corregir el data drifting, ajustando por algunos indices financieros. Utilizo Data Drifting de codigo de Analista Senior usando data set gerencial

Ajusto cantidad de meses a data set gerencial y vectores correspondientes de IPC, dolar blue, oficial y UVA

In [ ]:
# meses que me interesan para el ajuste de variables monetarias
vfoto_mes <- c( 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107, 202108, 202109
)


In [ ]:
# los valores que siguen fueron calculados por alumnos

# momento 1.0  31-dic-2020 a las 23:59
vIPC <- c(
  1.2118694724, 1.1881073259, 1.1693969743, 1.1375456949,
  1.1065619600, 1.0681100000, 1.0370000000, 1.0000000000,
  0.9680542110, 0.9344152616, 0.8882274350, 0.8532444140,
  0.8251880213, 0.8003764000, 0.7763107219, 0.7566381305,
  0.7289384687
)

vdolar_blue <- c(
  126.236842, 125.857143, 130.782609, 133.400000,
  137.954545, 170.619048, 160.400000, 153.052632,
  157.900000, 149.780952, 143.615385, 146.250000,
  153.550000, 162.000000, 178.478261, 180.878788,
  184.357143
)

vdolar_oficial <- c(
  70.047895, 72.520952, 75.324286, 77.488500,
  79.430909, 83.134762, 85.484737, 88.181667,
  91.474000, 93.997778, 96.635909, 98.526000,
  99.613158, 100.619048, 101.619048, 102.569048,
  103.781818
)

vUVA <- c(
  1.1770808941405335, 1.1570338657445522, 1.1388769475653255,
  1.1156993751209352, 1.093638313080772, 1.0657171590878205,
  1.0362173587708712, 1.0, 0.9669867858358365,
  0.9323750098728378, 0.8958202912590305, 0.8631993702994263,
  0.8253893405524657, 0.7928918905364516, 0.7666323845128089,
  0.7428976357662823, 0.721615762047849
)


In [ ]:
tb_indices <- as.data.table( list(
  "IPC" = vIPC,
  "dolar_blue" = vdolar_blue,
  "dolar_oficial" = vdolar_oficial,
  "UVA" = vUVA
  )
)

tb_indices[[ 'foto_mes' ]] <- vfoto_mes

tb_indices

IPC,dolar_blue,dolar_oficial,UVA,foto_mes
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1.2118695,126.2368,70.04789,1.1770809,202005
1.1881073,125.8571,72.52095,1.1570339,202006
1.1693970,130.7826,75.32429,1.1388769,202007
1.1375457,133.4000,77.48850,1.1156994,202008
1.1065620,137.9545,79.43091,1.0936383,202009
1.0681100,170.6190,83.13476,1.0657172,202010
1.0370000,160.4000,85.48474,1.0362174,202011
1.0000000,153.0526,88.18167,1.0000000,202012
0.9680542,157.9000,91.47400,0.9669868,202101


In [ ]:
drift_UVA <- function(campos_monetarios) {
  cat( "inicio drift_UVA()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.UVA,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_UVA()\n")
}


In [ ]:
drift_dolar_oficial <- function(campos_monetarios) {
  cat( "inicio drift_dolar_oficial()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_oficial,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_oficial()\n")
}


In [ ]:
drift_dolar_blue <- function(campos_monetarios) {
  cat( "inicio drift_dolar_blue()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_blue,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_blue()\n")
}


In [ ]:
drift_deflacion <- function(campos_monetarios) {
  cat( "inicio drift_deflacion()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.IPC,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_deflacion()\n")
}


In [ ]:
drift_rank_simple <- function(campos_drift) {

  cat( "inicio drift_rank_simple()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_rank") :=
      (frank(get(campo), ties.method = "random") - 1) / (.N - 1), by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat( "fin drift_rank_simple()\n")
}


In [ ]:
# El cero se transforma en cero
# los positivos se rankean por su lado
# los negativos se rankean por su lado

drift_rank_cero_fijo <- function(campos_drift) {

  cat( "inicio drift_rank_cero_fijo()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[get(campo) == 0, paste0(campo, "_rank") := 0]
    dataset[get(campo) > 0, paste0(campo, "_rank") :=
      frank(get(campo), ties.method = "random") / .N, by = list(foto_mes)]

    dataset[get(campo) < 0, paste0(campo, "_rank") :=
      -frank(-get(campo), ties.method = "random") / .N, by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat("\n")
  cat( "fin drift_rank_cero_fijo()\n")
}


In [ ]:
drift_estandarizar <- function(campos_drift) {

  cat( "inicio drift_estandarizar()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_normal") :=
      (get(campo) -mean(campo, na.rm=TRUE)) / sd(get(campo), na.rm=TRUE),
      by = list(foto_mes)]

    dataset[, (campo) := NULL]
  }
  cat( "fin drift_estandarizar()\n")
}


In [ ]:
# por como armé los nombres de campos,
#  estos son los campos que expresan variables monetarias
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like%
  "^(m|Visa_m|Master_m|vm_m)"]

campos_monetarios

[1] "mrentabilidad"           "mrentabilidad_annual"   
 [3] "mcomisiones"             "mactivos_margen"        
 [5] "mpasivos_margen"         "mcuenta_corriente"      
 [7] "mcaja_ahorro"            "mcuentas_saldo"         
 [9] "mtarjeta_visa_consumo"   "mtarjeta_master_consumo"
[11] "mprestamos_personales"   "mpayroll"               
[13] "Master_mpagominimo"      "Visa_mpagominimo"

In [ ]:
# ejecuto el Data Drifting
setorder( dataset, numero_de_cliente, foto_mes )


PARAM$DR$metodo <- "deflacion"

switch(PARAM$DR$metodo,
  "ninguno"        = cat("No hay correccion del data drifting"),
  "rank_simple"    = drift_rank_simple(campos_monetarios),
  "rank_cero_fijo" = drift_rank_cero_fijo(campos_monetarios),
  "deflacion"      = drift_deflacion(campos_monetarios),
  "dolar_blue"     = drift_dolarblue(campos_monetarios),
  "dolar_oficial"  = drift_dolaroficial(campos_monetarios),
  "UVA"            = drift_UVA(campos_monetarios),
  "estandarizar"   = drift_estandarizar(campos_monetarios)
)


inicio drift_deflacion()
fin drift_deflacion()


In [ ]:
colnames(dataset)

[1] "numero_de_cliente"           "foto_mes"                   
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"

#### 9.7.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia. Aplico codigo de analista Senior usando data set gerencial.

In [ ]:
if( !require("lubridate")) install.packages("lubridate", repos = "http://cran.us.r-project.org")
require("lubridate")

Loading required package: lubridate


Attaching package: ‘lubridate’


The following objects are masked from ‘package:data.table’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union




In [ ]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

In [ ]:
# Esta es la parte que los alumnos deben desplegar todo su ingenio
# Agregar aqui sus PROPIAS VARIABLES manuales

AgregarVariables_IntraMes <- function(dataset) {
  cat( "inicio AgregarVariables_IntraMes()\n")
  gc(verbose= FALSE)
  # INICIO de la seccion donde se deben hacer cambios con variables nuevas

  # el mes 1,2, ..12
  if( atributos_presentes( c("foto_mes") ))
    dataset[, kmes := foto_mes %% 100]

  # creo un ctr_quarter que tenga en cuenta cuando
  # los clientes hace 3 menos meses que estan
  # ya que seria injusto considerar las transacciones medidas en menor tiempo
  if( atributos_presentes( c("ctrx_quarter") ))
    dataset[, ctrx_quarter_normalizado := as.numeric(ctrx_quarter) ]

  if( atributos_presentes( c("ctrx_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 1, ctrx_quarter_normalizado := ctrx_quarter * 5]

  if( atributos_presentes( c("ctrx_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 2, ctrx_quarter_normalizado := ctrx_quarter * 2]

  if( atributos_presentes( c("ctrx_quarter", "cliente_antiguedad") ))
    dataset[
      cliente_antiguedad == 3,
      ctrx_quarter_normalizado := ctrx_quarter * 1.2
    ]

   if(atributos_presentes(c("foto_mes")))
    dataset[,foto_mes_formato_fecha := as.Date(paste(substr(dataset$foto_mes,1,4),substr(dataset$foto_mes,5,6),"01",sep='-'))]

  #dataset$foto_mes_formato_fecha <<- as.Date(paste(substr(dataset$foto_mes,1,4),substr(dataset$foto_mes,5,6),"01",sep='-'))

  if(atributos_presentes(c("cantidad_total_transacciones"))){
   auxiliarmenos1 <- dataset[,list(numero_de_cliente,foto_mes_formato_fecha, cantidad_total_transacciones)]
   auxiliarmenos2 <- dataset[,list(numero_de_cliente,foto_mes_formato_fecha,cantidad_total_transacciones)]
   # auxiliarmenos1$foto_mes_formato_fecha <- as.Date(auxiliarmenos1$foto_mes_formato_fecha)
   # auxiliarmenos2$foto_mes_formato_fecha <- as.Date(auxiliarmenos2$foto_mes_formato_fecha)
   auxiliarmenos1$foto_mes_formato_fecha <- auxiliarmenos1$foto_mes_formato_fecha  %m-%  months(1)
   auxiliarmenos2$foto_mes_formato_fecha <- auxiliarmenos2$foto_mes_formato_fecha %m-% months(2)
   auxiliarmenos1$codigo <- paste(auxiliarmenos1$numero_de_cliente,auxiliarmenos1$foto_mes_formato_fecha,sep='-')
   auxiliarmenos2$codigo <- paste(auxiliarmenos2$numero_de_cliente,auxiliarmenos2$foto_mes_formato_fecha,sep='-')

   dataset[, codigo := paste(numero_de_cliente, foto_mes_formato_fecha, sep='-') ]

   dataset[ auxiliarmenos1,
            on = "codigo",
            transaccionesmenos1 := i.cantidad_total_transacciones ]

   dataset[ auxiliarmenos2,
            on = "codigo",
            transaccionesmenos2 := i.cantidad_total_transacciones ]

   dataset[, cantidad_total_transacciones_quarter := rowSums(cbind(cantidad_total_transacciones +
    transaccionesmenos1 + transaccionesmenos2),na.rm=T) ]

   dataset[, codigo := NULL ]
   dataset[, transaccionesmenos1 := NULL ]
   dataset[, transaccionesmenos2 := NULL ]
   dataset[, foto_mes_formato_fecha := NULL ]
   rm(auxiliarmenos1)
   rm(auxiliarmenos2)
  }

  if( atributos_presentes( c("cantidad_total_transacciones_quarter") ))
    dataset[, cantidad_total_transacciones_quarter_normalizado := cantidad_total_transacciones_quarter]

  if( atributos_presentes( c("cantidad_total_transacciones_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 1, cantidad_total_transacciones_quarter_normalizado := cantidad_total_transacciones_quarter * 5]

  if( atributos_presentes( c("cantidad_total_transacciones_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 2, cantidad_total_transacciones_quarter_normalizado := cantidad_total_transacciones_quarter * 2]

  if( atributos_presentes( c("cantidad_total_transacciones_quarter", "cliente_antiguedad") ))
    dataset[cliente_antiguedad == 3, cantidad_total_transacciones_quarter_normalizado := cantidad_total_transacciones_quarter * 1.2]

  # variable extraida de una tesis de maestria de Irlanda
  if( atributos_presentes( c("mpayroll", "cliente_edad") ))
    dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

  # se crean los nuevos campos para MasterCard  y Visa,
  #  teniendo en cuenta los NA's
  # varias formas de combinar Visa_status y Master_status
  if( atributos_presentes( c("Master_status", "Visa_status") ))
  {
    dataset[, vm_status01 := pmax(Master_status, Visa_status, na.rm = TRUE)]
    dataset[, vm_status02 := Master_status + Visa_status]

    dataset[, vm_status03 := pmax(
      ifelse(is.na(Master_status), 10, Master_status),
      ifelse(is.na(Visa_status), 10, Visa_status)
    )]

    dataset[, vm_status04 := ifelse(is.na(Master_status), 10, Master_status)
      + ifelse(is.na(Visa_status), 10, Visa_status)]

    dataset[, vm_status05 := ifelse(is.na(Master_status), 10, Master_status)
      + 100 * ifelse(is.na(Visa_status), 10, Visa_status)]

    dataset[, vm_status06 := ifelse(is.na(Visa_status),
      ifelse(is.na(Master_status), 10, Master_status),
      Visa_status
    )]

    dataset[, mv_status07 := ifelse(is.na(Master_status),
      ifelse(is.na(Visa_status), 10, Visa_status),
      Master_status
    )]
  }


  # combino MasterCard y Visa
  if( atributos_presentes( c("Master_mfinanciacion_limite", "Visa_mfinanciacion_limite") ))
    dataset[, vm_mfinanciacion_limite := rowSums(cbind(Master_mfinanciacion_limite, Visa_mfinanciacion_limite), na.rm = TRUE)]

  if( atributos_presentes( c("Master_Fvencimiento", "Visa_Fvencimiento") ))
    dataset[, vm_Fvencimiento := pmin(Master_Fvencimiento, Visa_Fvencimiento, na.rm = TRUE)]

  if( atributos_presentes( c("Master_Finiciomora", "Visa_Finiciomora") ))
    dataset[, vm_Finiciomora := pmin(Master_Finiciomora, Visa_Finiciomora, na.rm = TRUE)]

  if( atributos_presentes( c("Master_msaldototal", "Visa_msaldototal") ))
    dataset[, vm_msaldototal := rowSums(cbind(Master_msaldototal, Visa_msaldototal), na.rm = TRUE)]

  if( atributos_presentes( c("Master_msaldopesos", "Visa_msaldopesos") ))
    dataset[, vm_msaldopesos := rowSums(cbind(Master_msaldopesos, Visa_msaldopesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_msaldodolares", "Visa_msaldodolares") ))
    dataset[, vm_msaldodolares := rowSums(cbind(Master_msaldodolares, Visa_msaldodolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mconsumospesos", "Visa_mconsumospesos") ))
    dataset[, vm_mconsumospesos := rowSums(cbind(Master_mconsumospesos, Visa_mconsumospesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mconsumosdolares", "Visa_mconsumosdolares") ))
    dataset[, vm_mconsumosdolares := rowSums(cbind(Master_mconsumosdolares, Visa_mconsumosdolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mlimitecompra", "Visa_mlimitecompra") ))
    dataset[, vm_mlimitecompra := rowSums(cbind(Master_mlimitecompra, Visa_mlimitecompra), na.rm = TRUE)]

  if( atributos_presentes( c("Master_madelantopesos", "Visa_madelantopesos") ))
    dataset[, vm_madelantopesos := rowSums(cbind(Master_madelantopesos, Visa_madelantopesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_madelantodolares", "Visa_madelantodolares") ))
    dataset[, vm_madelantodolares := rowSums(cbind(Master_madelantodolares, Visa_madelantodolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_fultimo_cierre", "Visa_fultimo_cierre") ))
    dataset[, vm_fultimo_cierre := pmax(Master_fultimo_cierre, Visa_fultimo_cierre, na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagado", "Visa_mpagado") ))
    dataset[, vm_mpagado := rowSums(cbind(Master_mpagado, Visa_mpagado), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagospesos", "Visa_mpagospesos") ))
    dataset[, vm_mpagospesos := rowSums(cbind(Master_mpagospesos, Visa_mpagospesos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagosdolares", "Visa_mpagosdolares") ))
    dataset[, vm_mpagosdolares := rowSums(cbind(Master_mpagosdolares, Visa_mpagosdolares), na.rm = TRUE)]

  if( atributos_presentes( c("Master_fechaalta", "Visa_fechaalta") ))
    dataset[, vm_fechaalta := pmax(Master_fechaalta, Visa_fechaalta, na.rm = TRUE)]

  if( atributos_presentes( c("Master_mconsumototal", "Visa_mconsumototal") ))
    dataset[, vm_mconsumototal := rowSums(cbind(Master_mconsumototal, Visa_mconsumototal), na.rm = TRUE)]

  if( atributos_presentes( c("Master_cconsumos", "Visa_cconsumos") ))
    dataset[, vm_cconsumos := rowSums(cbind(Master_cconsumos, Visa_cconsumos), na.rm = TRUE)]

  if( atributos_presentes( c("Master_cadelantosefectivo", "Visa_cadelantosefectivo") ))
    dataset[, vm_cadelantosefectivo := rowSums(cbind(Master_cadelantosefectivo, Visa_cadelantosefectivo), na.rm = TRUE)]

  if( atributos_presentes( c("Master_mpagominimo", "Visa_mpagominimo") ))
    dataset[, vm_mpagominimo := rowSums(cbind(Master_mpagominimo, Visa_mpagominimo), na.rm = TRUE)]

  # a partir de aqui juego con la suma de Mastercard y Visa
  if( atributos_presentes( c("Master_mlimitecompra", "vm_mlimitecompra") ))
    dataset[, vmr_Master_mlimitecompra := Master_mlimitecompra / vm_mlimitecompra]

  if( atributos_presentes( c("Visa_mlimitecompra", "vm_mlimitecompra") ))
    dataset[, vmr_Visa_mlimitecompra := Visa_mlimitecompra / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldototal", "vm_mlimitecompra") ))
    dataset[, vmr_msaldototal := vm_msaldototal / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldopesos", "vm_mlimitecompra") ))
    dataset[, vmr_msaldopesos := vm_msaldopesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldopesos", "vm_msaldototal") ))
    dataset[, vmr_msaldopesos2 := vm_msaldopesos / vm_msaldototal]

  if( atributos_presentes( c("vm_msaldodolares", "vm_mlimitecompra") ))
    dataset[, vmr_msaldodolares := vm_msaldodolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_msaldodolares", "vm_msaldototal") ))
    dataset[, vmr_msaldodolares2 := vm_msaldodolares / vm_msaldototal]

  if( atributos_presentes( c("vm_mconsumospesos", "vm_mlimitecompra") ))
    dataset[, vmr_mconsumospesos := vm_mconsumospesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mconsumosdolares", "vm_mlimitecompra") ))
    dataset[, vmr_mconsumosdolares := vm_mconsumosdolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_madelantopesos", "vm_mlimitecompra") ))
    dataset[, vmr_madelantopesos := vm_madelantopesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_madelantodolares", "vm_mlimitecompra") ))
    dataset[, vmr_madelantodolares := vm_madelantodolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagado", "vm_mlimitecompra") ))
    dataset[, vmr_mpagado := vm_mpagado / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagospesos", "vm_mlimitecompra") ))
    dataset[, vmr_mpagospesos := vm_mpagospesos / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagosdolares", "vm_mlimitecompra") ))
    dataset[, vmr_mpagosdolares := vm_mpagosdolares / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mconsumototal", "vm_mlimitecompra") ))
    dataset[, vmr_mconsumototal := vm_mconsumototal / vm_mlimitecompra]

  if( atributos_presentes( c("vm_mpagominimo", "vm_mlimitecompra") ))
    dataset[, vmr_mpagominimo := vm_mpagominimo / vm_mlimitecompra]

  # Aqui debe usted agregar sus propias nuevas variables

  # valvula de seguridad para evitar valores infinitos
  # paso los infinitos a NULOS
  infinitos <- lapply(
    names(dataset),
    function(.name) dataset[, sum(is.infinite(get(.name)))]
  )

  infinitos_qty <- sum(unlist(infinitos))
  if (infinitos_qty > 0) {
    cat(
      "ATENCION, hay", infinitos_qty,
      "valores infinitos en tu dataset. Seran pasados a NA\n"
    )
    dataset[mapply(is.infinite, dataset)] <<- NA
  }


  # valvula de seguridad para evitar valores NaN  que es 0/0
  # paso los NaN a 0 , decision polemica si las hay
  # se invita a asignar un valor razonable segun la semantica del campo creado
  nans <- lapply(
    names(dataset),
    function(.name) dataset[, sum(is.nan(get(.name)))]
  )

  nans_qty <- sum(unlist(nans))
  if (nans_qty > 0) {
    cat(
      "ATENCION, hay", nans_qty,
      "valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0\n"
    )

    cat("Si no te gusta la decision, modifica a gusto el programa!\n\n")
    dataset[mapply(is.nan, dataset)] <<- 0
  }

  cat( "fin AgregarVariables_IntraMes()\n")
}


In [ ]:
# agrego las variables intra mes

AgregarVariables_IntraMes(dataset)

inicio AgregarVariables_IntraMes()
ATENCION, hay 8 valores NaN 0/0 en tu dataset. Seran pasados arbitrariamente a 0
Si no te gusta la decision, modifica a gusto el programa!

fin AgregarVariables_IntraMes()


In [ ]:
# visualizo las columas del dataset a esta etapa
ncol(dataset)
colnames(dataset)

[1] 45

[1] "numero_de_cliente"           "foto_mes"                   
 [3] "internet"                    "cliente_edad"               
 [5] "cliente_antiguedad"          "mrentabilidad"              
 [7] "mrentabilidad_annual"        "mcomisiones"                
 [9] "mactivos_margen"             "mpasivos_margen"            
[11] "cproductos"                  "mcuenta_corriente"          
[13] "mcaja_ahorro"                "cdescubierto_preacordado"   
[15] "mcuentas_saldo"              "ctarjeta_visa_transacciones"
[17] "mtarjeta_visa_consumo"       "mtarjeta_master_consumo"    
[19] "mprestamos_personales"       "cpayroll_trx"               
[21] "mpayroll"                    "ccomisiones_mantenimiento"  
[23] "ccallcenter_transacciones"   "chomebanking_transacciones" 
[25] "ctrx_quarter"                "Master_status"              
[27] "Master_fechaalta"            "Master_mpagominimo"         
[29] "Visa_status"                 "Visa_fechaalta"             
[31] "Visa_mpagominimo"            "clase_ternaria"             
[33] "kmes"                        "ctrx_quarter_normalizado"   
[35] "foto_mes_formato_fecha"      "mpayroll_sobre_edad"        
[37] "vm_status01"                 "vm_status02"                
[39] "vm_status03"                 "vm_status04"                
[41] "vm_status05"                 "vm_status06"                
[43] "mv_status07"                 "vm_fechaalta"               
[45] "vm_mpagominimo"

#### 9.7.1.4  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

Uso codigo de Analista Senior con data set gerencial

In [ ]:
if( !require("Rcpp")) install.packages("Rcpp", repos = "http://cran.us.r-project.org")
require("Rcpp")

Loading required package: Rcpp



In [ ]:
# se calculan para los 6 meses previos el minimo, maximo y
#  tendencia calculada con cuadrados minimos
# la formula de calculo de la tendencia puede verse en
#  https://stats.libretexts.org/Bookshelves/Introductory_Statistics/Book%3A_Introductory_Statistics_(Shafer_and_Zhang)/10%3A_Correlation_and_Regression/10.04%3A_The_Least_Squares_Regression_Line
# para la maxíma velocidad esta funcion esta escrita en lenguaje C,
# y no en la porqueria de R o Python

cppFunction("NumericVector fhistC(NumericVector pcolumna, IntegerVector pdesde )
{
  /* Aqui se cargan los valores para la regresion */
  double  x[100] ;
  double  y[100] ;

  int n = pcolumna.size();
  NumericVector out( 5*n );

  for(int i = 0; i < n; i++)
  {
    //lag
    if( pdesde[i]-1 < i )  out[ i + 4*n ]  =  pcolumna[i-1] ;
    else                   out[ i + 4*n ]  =  NA_REAL ;


    int  libre    = 0 ;
    int  xvalor   = 1 ;

    for( int j= pdesde[i]-1;  j<=i; j++ )
    {
       double a = pcolumna[j] ;

       if( !R_IsNA( a ) )
       {
          y[ libre ]= a ;
          x[ libre ]= xvalor ;
          libre++ ;
       }

       xvalor++ ;
    }

    /* Si hay al menos dos valores */
    if( libre > 1 )
    {
      double  xsum  = x[0] ;
      double  ysum  = y[0] ;
      double  xysum = xsum * ysum ;
      double  xxsum = xsum * xsum ;
      double  vmin  = y[0] ;
      double  vmax  = y[0] ;

      for( int h=1; h<libre; h++)
      {
        xsum  += x[h] ;
        ysum  += y[h] ;
        xysum += x[h]*y[h] ;
        xxsum += x[h]*x[h] ;

        if( y[h] < vmin )  vmin = y[h] ;
        if( y[h] > vmax )  vmax = y[h] ;
      }

      out[ i ]  =  (libre*xysum - xsum*ysum)/(libre*xxsum -xsum*xsum) ;
      out[ i + n ]    =  vmin ;
      out[ i + 2*n ]  =  vmax ;
      out[ i + 3*n ]  =  ysum / libre ;
    }
    else
    {
      out[ i       ]  =  NA_REAL ;
      out[ i + n   ]  =  NA_REAL ;
      out[ i + 2*n ]  =  NA_REAL ;
      out[ i + 3*n ]  =  NA_REAL ;
    }
  }

  return  out;
}")


In [ ]:
# calcula la tendencia de las variables cols de los ultimos 6 meses
# la tendencia es la pendiente de la recta que ajusta por cuadrados minimos
# La funcionalidad de ratioavg es autoria de  Daiana Sparta,  UAustral  2021

TendenciaYmuchomas <- function(
    dataset, cols, ventana = 6, tendencia = TRUE,
    minimo = TRUE, maximo = TRUE, promedio = TRUE,
    ratioavg = FALSE, ratiomax = FALSE) {
  gc(verbose= FALSE)
  # Esta es la cantidad de meses que utilizo para la historia
  ventana_regresion <- ventana

  last <- nrow(dataset)

  # creo el vector_desde que indica cada ventana
  # de esta forma se acelera el procesamiento ya que lo hago una sola vez
  vector_ids <- dataset[ , numero_de_cliente ]

  vector_desde <- seq(
    -ventana_regresion + 2,
    nrow(dataset) - ventana_regresion + 1
  )

  vector_desde[1:ventana_regresion] <- 1

  for (i in 2:last) {
    if (vector_ids[i - 1] != vector_ids[i]) {
      vector_desde[i] <- i
    }
  }
  for (i in 2:last) {
    if (vector_desde[i] < vector_desde[i - 1]) {
      vector_desde[i] <- vector_desde[i - 1]
    }
  }

  for (campo in cols) {
    nueva_col <- fhistC(dataset[, get(campo)], vector_desde)

    if (tendencia) {
      dataset[, paste0(campo, "_tend", ventana) :=
        nueva_col[(0 * last + 1):(1 * last)]]
    }

    if (minimo) {
      dataset[, paste0(campo, "_min", ventana) :=
        nueva_col[(1 * last + 1):(2 * last)]]
    }

    if (maximo) {
      dataset[, paste0(campo, "_max", ventana) :=
        nueva_col[(2 * last + 1):(3 * last)]]
    }

    if (promedio) {
      dataset[, paste0(campo, "_avg", ventana) :=
        nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratioavg) {
      dataset[, paste0(campo, "_ratioavg", ventana) :=
        get(campo) / nueva_col[(3 * last + 1):(4 * last)]]
    }

    if (ratiomax) {
      dataset[, paste0(campo, "_ratiomax", ventana) :=
        get(campo) / nueva_col[(2 * last + 1):(3 * last)]]
    }
  }
}


In [ ]:
# Feature Engineering Historico

setorder(dataset, numero_de_cliente, foto_mes)

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


In [ ]:
# parametros de Feature Engineering Historico de Tendencias
PARAM$FE_hist$Tendencias$run <- TRUE
PARAM$FE_hist$Tendencias$ventana <- 6
PARAM$FE_hist$Tendencias$tendencia <- TRUE
PARAM$FE_hist$Tendencias$minimo <- TRUE
PARAM$FE_hist$Tendencias$maximo <- TRUE
PARAM$FE_hist$Tendencias$promedio <- TRUE
PARAM$FE_hist$Tendencias$ratioavg <- FALSE
PARAM$FE_hist$Tendencias$ratiomax <- FALSE


In [ ]:
cols_lagueables <- intersect(cols_lagueables, colnames(dataset))
setorder(dataset, numero_de_cliente, foto_mes)

if( PARAM$FE_hist$Tendencias$run) {
    TendenciaYmuchomas(dataset,
    cols = cols_lagueables,
    ventana = PARAM$FE_hist$Tendencias$ventana, # 6 meses de historia
    tendencia = PARAM$FE_hist$Tendencias$tendencia,
    minimo = PARAM$FE_hist$Tendencias$minimo,
    maximo = PARAM$FE_hist$Tendencias$maximo,
    promedio = PARAM$FE_hist$Tendencias$promedio,
    ratioavg = PARAM$FE_hist$Tendencias$ratioavg,
    ratiomax = PARAM$FE_hist$Tendencias$ratiomax
  )
}



Verificacion de los campos recien creados

In [ ]:
ncol(dataset)
colnames(dataset)

[1] 255

[1] "numero_de_cliente"                  "foto_mes"                          
  [3] "internet"                           "cliente_edad"                      
  [5] "cliente_antiguedad"                 "mrentabilidad"                     
  [7] "mrentabilidad_annual"               "mcomisiones"                       
  [9] "mactivos_margen"                    "mpasivos_margen"                   
 [11] "cproductos"                         "mcuenta_corriente"                 
 [13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
 [15] "mcuentas_saldo"                     "ctarjeta_visa_transacciones"       
 [17] "mtarjeta_visa_consumo"              "mtarjeta_master_consumo"           
 [19] "mprestamos_personales"              "cpayroll_trx"                      
 [21] "mpayroll"                           "ccomisiones_mantenimiento"         
 [23] "ccallcenter_transacciones"          "chomebanking_transacciones"        
 [25] "ctrx_quarter"                       "Master_status"                     
 [27] "Master_fechaalta"                   "Master_mpagominimo"                
 [29] "Visa_status"                        "Visa_fechaalta"                    
 [31] "Visa_mpagominimo"                   "clase_ternaria"                    
 [33] "kmes"                               "ctrx_quarter_normalizado"          
 [35] "foto_mes_formato_fecha"             "mpayroll_sobre_edad"               
 [37] "vm_status01"                        "vm_status02"                       
 [39] "vm_status03"                        "vm_status04"                       
 [41] "vm_status05"                        "vm_status06"                       
 [43] "mv_status07"                        "vm_fechaalta"                      
 [45] "vm_mpagominimo"                     "internet_lag1"                     
 [47] "cliente_edad_lag1"                  "cliente_antiguedad_lag1"           
 [49] "mrentabilidad_lag1"                 "mrentabilidad_annual_lag1"         
 [51] "mcomisiones_lag1"                   "mactivos_margen_lag1"              
 [53] "mpasivos_margen_lag1"               "cproductos_lag1"                   
 [55] "mcuenta_corriente_lag1"             "mcaja_ahorro_lag1"                 
 [57] "cdescubierto_preacordado_lag1"      "mcuentas_saldo_lag1"               
 [59] "ctarjeta_visa_transacciones_lag1"   "mtarjeta_visa_consumo_lag1"        
 [61] "mtarjeta_master_consumo_lag1"       "mprestamos_personales_lag1"        
 [63] "cpayroll_trx_lag1"                  "mpayroll_lag1"                     
 [65] "ccomisiones_mantenimiento_lag1"     "ccallcenter_transacciones_lag1"    
 [67] "chomebanking_transacciones_lag1"    "ctrx_quarter_lag1"                 
 [69] "Master_status_lag1"                 "Master_fechaalta_lag1"             
 [71] "Master_mpagominimo_lag1"            "Visa_status_lag1"                  
 [73] "Visa_fechaalta_lag1"                "Visa_mpagominimo_lag1"             
 [75] "kmes_lag1"                          "ctrx_quarter_normalizado_lag1"     
 [77] "foto_mes_formato_fecha_lag1"        "mpayroll_sobre_edad_lag1"          
 [79] "vm_status01_lag1"                   "vm_status02_lag1"                  
 [81] "vm_status03_lag1"                   "vm_status04_lag1"                  
 [83] "vm_status05_lag1"                   "vm_status06_lag1"                  
 [85] "mv_status07_lag1"                   "vm_fechaalta_lag1"                 
 [87] "vm_mpagominimo_lag1"                "internet_lag2"                     
 [89] "cliente_edad_lag2"                  "cliente_antiguedad_lag2"           
 [91] "mrentabilidad_lag2"                 "mrentabilidad_annual_lag2"         
 [93] "mcomisiones_lag2"                   "mactivos_margen_lag2"              
 [95] "mpasivos_margen_lag2"               "cproductos_lag2"                   
 [97] "mcuenta_corriente_lag2"             "mcaja_ahorro_lag2"                 
 [99] "cdescubierto_preacordado_lag2"      "mcuentas_saldo_lag2"               
[1

#### 9.7.1.5  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest



In [ ]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

Loading required package: lightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lightgbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: lightgbm



In [ ]:
AgregaVarRandomForest <- function() {

  cat( "inicio AgregaVarRandomForest()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$FE_rf$train$clase01_valor1,
      clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c( "clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$FE_rf$train$training )]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
     data = dtrain,
     param = PARAM$FE_rf$lgb_param,
     verbose = -100
  )

  cat( "Fin construccion RandomForest\n" )
  # grabo el modelo, achivo .model
  lgb.save(modelo, file="modelo.model" )

  qarbolitos <- copy(PARAM$FE_rf$lgb_param$num_iterations)

  periodos <- dataset[ , unique( foto_mes ) ]

  for( periodo in  periodos )
  {
    cat( "periodo = ", periodo, "\n" )
    datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

    cat( "Inicio prediccion\n" )
    prediccion <- predict(
        modelo,
        datamatrix,
        type = "leaf"
    )
    cat( "Fin prediccion\n" )

    for( arbolito in 1:qarbolitos )
    {
       cat( arbolito, " " )
       hojas_arbol <- unique(prediccion[ , arbolito])

       for (pos in 1:length(hojas_arbol)) {
         # el numero de nodo de la hoja, estan salteados
         nodo_id <- hojas_arbol[pos]
         dataset[ foto_mes== periodo, paste0(
            "rf_", sprintf("%03d", arbolito),
             "_", sprintf("%03d", nodo_id)
          ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

       }

       rm( hojas_arbol )
    }
    cat( "\n" )

    rm( prediccion )
    rm( datamatrix )
    gc(verbose= FALSE)
  }

  gc(verbose= FALSE)

  # borro clase01 , no debe ensuciar el dataset
  dataset[ , clase01 := NULL ]

}


Dejo PARAM$FE_rf$train$training igual ya que tengo los meses del 202101, 202102, 202103  en el data set Gerencial

In [ ]:
# Parametros de Feature Engineering  a partir de hojas de Random Forest

# Estos CUATRO parametros son los que se deben modificar
PARAM$FE_rf$arbolitos= 20
PARAM$FE_rf$hojas_por_arbol= 16
PARAM$FE_rf$datos_por_hoja= 100
PARAM$FE_rf$mtry_ratio= 0.2

# Estos son quasi fijos
PARAM$FE_rf$train$clase01_valor1 <- c( "BAJA+2", "BAJA+1")
PARAM$FE_rf$train$training <- c( 202101, 202102, 202103)
#Dejo estos meses ya que en gerencial los tengo

# Estos TAMBIEN son quasi fijos
PARAM$FE_rf$lgb_param <-list(
    # parametros que se pueden cambiar
    num_iterations = PARAM$FE_rf$arbolitos,
    num_leaves  = PARAM$FE_rf$hojas_por_arbol,
    min_data_in_leaf = PARAM$FE_rf$datos_por_hoja,
    feature_fraction_bynode  = PARAM$FE_rf$mtry_ratio,

    # para que LightGBM emule Random Forest
    boosting = "rf",
    bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
    bagging_freq = 1.0,
    feature_fraction = 1.0,

    # genericos de LightGBM
    max_bin = 31L,
    objective = "binary",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    force_row_wise = TRUE,
    verbosity = -100,
    max_depth = -1L,
    min_gain_to_split = 0.0,
    min_sum_hessian_in_leaf = 0.001,
    lambda_l1 = 0.0,
    lambda_l2 = 0.0,

    pos_bagging_fraction = 1.0,
    neg_bagging_fraction = 1.0,
    is_unbalance = FALSE,
    scale_pos_weight = 1.0,

    drop_rate = 0.1,
    max_drop = 50,
    skip_drop = 0.5,

    extra_trees = FALSE
  )

In [ ]:
# Feature Engineering agregando variables de Random Forest
AgregaVarRandomForest()

inicio AgregaVarRandomForest()
Fin construccion RandomForest
periodo =  202005 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202006 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202007 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202008 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202009 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202010 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202011 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202012 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7

In [ ]:
ncol(dataset)
colnames(dataset)

[1] 576

[1] "numero_de_cliente"                  "foto_mes"                          
  [3] "internet"                           "cliente_edad"                      
  [5] "cliente_antiguedad"                 "mrentabilidad"                     
  [7] "mrentabilidad_annual"               "mcomisiones"                       
  [9] "mactivos_margen"                    "mpasivos_margen"                   
 [11] "cproductos"                         "mcuenta_corriente"                 
 [13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
 [15] "mcuentas_saldo"                     "ctarjeta_visa_transacciones"       
 [17] "mtarjeta_visa_consumo"              "mtarjeta_master_consumo"           
 [19] "mprestamos_personales"              "cpayroll_trx"                      
 [21] "mpayroll"                           "ccomisiones_mantenimiento"         
 [23] "ccallcenter_transacciones"          "chomebanking_transacciones"        
 [25] "ctrx_quarter"                       "Master_status"                     
 [27] "Master_fechaalta"                   "Master_mpagominimo"                
 [29] "Visa_status"                        "Visa_fechaalta"                    
 [31] "Visa_mpagominimo"                   "clase_ternaria"                    
 [33] "kmes"                               "ctrx_quarter_normalizado"          
 [35] "foto_mes_formato_fecha"             "mpayroll_sobre_edad"               
 [37] "vm_status01"                        "vm_status02"                       
 [39] "vm_status03"                        "vm_status04"                       
 [41] "vm_status05"                        "vm_status06"                       
 [43] "mv_status07"                        "vm_fechaalta"                      
 [45] "vm_mpagominimo"                     "internet_lag1"                     
 [47] "cliente_edad_lag1"                  "cliente_antiguedad_lag1"           
 [49] "mrentabilidad_lag1"                 "mrentabilidad_annual_lag1"         
 [51] "mcomisiones_lag1"                   "mactivos_margen_lag1"              
 [53] "mpasivos_margen_lag1"               "cproductos_lag1"                   
 [55] "mcuenta_corriente_lag1"             "mcaja_ahorro_lag1"                 
 [57] "cdescubierto_preacordado_lag1"      "mcuentas_saldo_lag1"               
 [59] "ctarjeta_visa_transacciones_lag1"   "mtarjeta_visa_consumo_lag1"        
 [61] "mtarjeta_master_consumo_lag1"       "mprestamos_personales_lag1"        
 [63] "cpayroll_trx_lag1"                  "mpayroll_lag1"                     
 [65] "ccomisiones_mantenimiento_lag1"     "ccallcenter_transacciones_lag1"    
 [67] "chomebanking_transacciones_lag1"    "ctrx_quarter_lag1"                 
 [69] "Master_status_lag1"                 "Master_fechaalta_lag1"             
 [71] "Master_mpagominimo_lag1"            "Visa_status_lag1"                  
 [73] "Visa_fechaalta_lag1"                "Visa_mpagominimo_lag1"             
 [75] "kmes_lag1"                          "ctrx_quarter_normalizado_lag1"     
 [77] "foto_mes_formato_fecha_lag1"        "mpayroll_sobre_edad_lag1"          
 [79] "vm_status01_lag1"                   "vm_status02_lag1"                  
 [81] "vm_status03_lag1"                   "vm_status04_lag1"                  
 [83] "vm_status05_lag1"                   "vm_status06_lag1"                  
 [85] "mv_status07_lag1"                   "vm_fechaalta_lag1"                 
 [87] "vm_mpagominimo_lag1"                "internet_lag2"                     
 [89] "cliente_edad_lag2"                  "cliente_antiguedad_lag2"           
 [91] "mrentabilidad_lag2"                 "mrentabilidad_annual_lag2"         
 [93] "mcomisiones_lag2"                   "mactivos_margen_lag2"              
 [95] "mpasivos_margen_lag2"               "cproductos_lag2"                   
 [97] "mcuenta_corriente_lag2"             "mcaja_ahorro_lag2"                 
 [99] "cdescubierto_preacordado_lag2"      "mcuentas_saldo_lag2"               
[1

#### 9.7.1.6  FEhist Reduccion dimensionalidad con canaritos

No aplico Canaritos

### 9.7.2 Modelado

#### 9.7.2.1 Training Strategy


Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 201901, 202107 ] sin undersampling de los CONTINUA

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 201901, 202105 ]  donde se consideran el 20% de los CONTINUA

   Intento usar la mayor cantidad de meses posibles del data set gerencial. NO elimino ningun mes por covid ni nada.

   Uso meses en base a data set gerencial

In [ ]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105
)

# Uso undersampling de los CONTINUA del 0,2.
# Me corre mucho mas rapido y me da ganancias similares

PARAM$trainingstrategy$training_pct <- 0.2


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [ ]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [ ]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [ ]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]

# ========================================
# ESTRATEGIA DE PESOS POR AÑO (OPTIMIZACIÓN BAYESIANA)
# ========================================
# Extraer el año de foto_mes
dataset[, anio := as.integer(substr(as.character(foto_mes), 1, 4))]

# Obtener años únicos ordenados (del más viejo al más reciente)
anios_unicos <- sort(unique(dataset$anio))

# Asignar pesos: año más viejo = 1, siguiente = 2, etc.
dataset[, peso_norm := match(anio, anios_unicos)]

# Verificar la asignación
print("Distribución de pesos por año en Optimización Bayesiana:")
print(dataset[, .(peso = unique(peso_norm)), by = anio][order(anio)])

if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  weight= dataset[fold_train == TRUE, peso_norm],  # Agrego los pesos aquí <------------------
  free_raw_data= TRUE
)

# ========================================
# ========================================

[1] "Distribución de pesos por año en Optimización Bayesiana:"
    anio  peso
   <int> <int>
1:  2020     1
2:  2021     2


In [ ]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

[1] 16551

####  9.7.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [ ]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘DiceKriging’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DiceKriging

Loading required package: mlrMBO

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘mlrMBO’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘XML’, ‘fastmatch’, ‘RcppArmadillo’, ‘mlr’, ‘ParamHelpers’, ‘smoof’, ‘BBmisc’, ‘checkmate’, ‘lhs’, ‘parallelMap’


Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package

Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [ ]:
# Aunque soy gerencial le dejo 40 ya que mi data set es mas chico (mas aun con undersampling
# del 0.2) y me corre rapido
PARAM$hipeparametertuning$num_interations <- 40

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  extra_trees = FALSE,

  max_depth = -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split = 0.0, # min_gain_to_split >= 0.0
  min_sum_hessian_in_leaf = 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1 = 0.0, # lambda_l1 >= 0.0
  lambda_l2 = 0.0, # lambda_l2 >= 0.0

  bagging_fraction = 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction = 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction = 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance = FALSE, #
  scale_pos_weight = 1.0, # scale_pos_weight > 0.0

  drop_rate = 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop = 50, # <=0 means no limit
  skip_drop = 0.5, # 0.0 <= skip_drop <= 1.0

  max_bin= 31,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)



Nuevos  pseudo hiperparámetros

*   leaf_size (0.0,  0.5]  es el   min_data_in_leaf / nrow(dataset)
*   coverage (0.0, 1.0]  se cumple   num_leaves =   coverage * (nrow(dataset)/min_data_in_leaf)



In [ ]:
# Notar que se usan los pseudo hiperparametros "ratio"  leaf_size  y coverage
#  que luego terminan en  min_data_in_leaf y  num_leaves
#  de forma que sea invariante al tamaño del dataset

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeNumericParam("learning_rate", lower = 0.02, upper = 0.3),
  makeNumericParam("feature_fraction", lower = 0.05, upper = 0.90),
  makeNumericParam("coverage", lower = 0.05, upper = 1.0), # nuevo
  makeNumericParam("leaf_size", lower = 0.001, upper = 0.2) # nuevo
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [ ]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # hago la transformacion de leaf_size y  coverage
  if( "leaf_size"  %in% names(param_completo) &
      "coverage"  %in% names(param_completo)
  )
  {
    # primero defino el tamaño de las hojas
    param_completo$min_data_in_leaf <- pmax( 1,  round( nrow(dtrain) * param_completo$leaf_size )  )
    # luego la cantidad de hojas en funcion del valor anterior, el coverage, y la cantidad de registros
    param_completo$num_leaves <- pmin( 131072,
      pmax( 8,  round( ( param_completo$coverage * nrow( dtrain ) / param_completo$min_data_in_leaf ) ) ))
  }

  if( "leaf_size"  %in% names(param_completo) &
      !( "coverage"  %in% names(param_completo) )
  )
  {
    # primero defino el tamaño de las hojas
    param_completo$min_data_in_leaf <- pmax( 1,  round( nrow(dtrain) * param_completo$leaf_size )  )
  }


  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [ ]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [ ]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Sat Nov 29 12:45:16 PM 2025 AUC 0.916478674819674

Sat Nov 29 12:45:45 PM 2025 AUC 0.919213631803216

Sat Nov 29 12:46:00 PM 2025 AUC 0.917747040518654

Sat Nov 29 12:46:41 PM 2025 AUC 0.91558787614782

Sat Nov 29 12:46:57 PM 2025 AUC 0.916723473570308

Sat Nov 29 12:47:11 PM 2025 AUC 0.917248522286766

Sat Nov 29 12:47:24 PM 2025 AUC 0.914987638416142

Sat Nov 29 12:47:59 PM 2025 AUC 0.913800717829181

Sat Nov 29 12:48:14 PM 2025 AUC 0.918594509927499

Sat Nov 29 12:48:30 PM 2025 AUC 0.908692730647228

Sat Nov 29 12:48:45 PM 2025 AUC 0.912709608206145

Sat Nov 29 12:49:05 PM 2025 AUC 0.909059754992713

Sat Nov 29 12:49:21 PM 2025 AUC 0.913345065445724

Sat Nov 29 12:49:37 PM 2025 AUC 0.918442509946035

Sat Nov 29 12:49:54 PM 2025 AUC 0.913729583691514

Sat Nov 29 12:50:23 PM 2025 AUC 0.921674131503155

[mbo] 0: learning_rate=0.189; feature_fraction=0.284; coverage=0.702; leaf_size=0.0958 : y = 0.916 : 20.2 secs : initdesign

[mbo] 0: le

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [ ]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

print(PARAM$out$lgbm$mejores_hiperparametros)

   learning_rate feature_fraction   coverage leaf_size num_iterations
           <num>            <num>      <num>     <num>          <int>
1:    0.08158456        0.3856678 0.08769214 0.0783128            153


### 9.7.3 Produccion

#### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

Uso bloque de codigo del Experimento 8 "Estrategias de Pesos por Año" de los compañeros Carlos Aular y Juan Pedrozo

uso meses que tengo en el data set gerencial

In [ ]:
PARAM$trainingstrategy$final_train <- c(
  202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]
# ========================================
# ESTRATEGIA DE PESOS POR AÑO
# ========================================
# Verificar si ya existe la columna peso_norm, si no, crearla
if (!("peso_norm" %in% colnames(dataset))) {
  # Extraer el año de foto_mes
  dataset[, anio := as.integer(substr(as.character(foto_mes), 1, 4))]

  # Obtener años únicos ordenados
  anios_unicos <- sort(unique(dataset$anio))

  # Asignar pesos
  dataset[, peso_norm := match(anio, anios_unicos)]
}

# Verificar la asignación
print("Distribución de pesos por año en Final Train:")
print(dataset[, .(peso = unique(peso_norm)), by = anio][order(anio)])

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  weight= dataset[fold_final_train == TRUE, peso_norm],  # Agrego los pesos aquí <------------------
  free_raw_data= TRUE
)


nrow( dfinal_train) # verifico el tamaño
# ========================================
# ========================================

[1] "Distribución de pesos por año en Final Train:"
    anio  peso
   <int> <int>
1:  2020     1
2:  2021     2


[1] 240562

##### Final Training Hyperparameters

In [ ]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

# hago la transformacion de leaf_size y  coverage
  if( "leaf_size"  %in% names(param_final) &
      "coverage"  %in% names(param_final)
  )
  {
    # primero defino el tamaño de las hojas
    param_final$min_data_in_leaf <- pmax( 1,  round( nrow(dtrain) * param_final$leaf_size )  )
    # luego la cantidad de hojas en funcion del valor anterior, el coverage, y la cantidad de registros
    param_final$num_leaves <- pmin( 131072,
      pmax( 8,  round( ( param_final$coverage * nrow( dtrain ) / param_final$min_data_in_leaf ) ) ))
  }

  if( "leaf_size"  %in% names(param_final) &
      !( "coverage"  %in% names(param_final) )
  )
  {
    # primero defino el tamaño de las hojas
    param_final$min_data_in_leaf <- pmax( 1,  round( nrow(dtrain) * param_final$leaf_size )  )
  }

In [ ]:
param_final

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] TRUE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$verbosity
[1] -100

$force_row_wise
[1] TRUE

$seed
[1] 100007

$extra_trees
[1] FALSE

$max_depth
[1] -1

$min_gain_to_split
[1] 0

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 0

$lambda_l2
[1] 0

$bagging_fraction
[1] 1

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$max_bin
[1] 31

$learning_rate
[1] 0.08158456

$feature_fraction
[1] 0.3856678

$coverage
[1] 0.08769214

$leaf_size
[1] 0.0783128

$num_iterations
[1] 153

$min_data_in_leaf
[1] 3358

$num_leaves
[1] 8

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

In [ ]:
PARAM$FT$semillerio <- 40  # cantidad de semillas

In [ ]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘primes’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: primes



In [ ]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$FT$semillas <- sample(primos)[seq(PARAM$FT$semillerio)]

cat( PARAM$FT$semillas)

116351 450529 558251 518153 693149 873319 263201 925033 542687 690607 952363 354799 987631 644051 969097 951479 283403 225289 343709 591653 291007 165707 633133 463753 561251 296563 201947 816157 372293 265883 379073 136481 253423 600983 553471 644327 321331 540587 480827 524731

In [ ]:
dir.create("modelos", showWarnings =FALSE)

primero <- TRUE
for( sem in PARAM$FT$semillas)
{
  nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
  if( !file.exists(nombre_arch) )
  {
    param_final$seed <- sem

    set.seed(sem, kind = "L'Ecuyer-CMRG")
    final_model <- lgb.train(
      data= dfinal_train,
      param= param_final,
      verbose= -100
    )

    lgb.save(final_model, nombre_arch) # grabo el modelo"

    # grabo la primer importancia de variables
    #  Natalia : da lo mismo cual se guarda
    if( primero)
    {
      primero <- FALSE
      tb_importancia <- as.data.table(lgb.importance(final_model))
      archivo_importancia <- "impo.txt"

      fwrite( tb_importancia,
        file= archivo_importancia,
        sep= "\t"
      )
    }
 }
}

#### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

In [ ]:
# aplico final_model   a dfuture

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := 0]

datos_matrix <- data.matrix(dfuture[, campos_buenos, with= FALSE])


for( isem in seq(length(PARAM$FT$semillas)) )
{
   sem <- PARAM$FT$semillas[ isem ]
   nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
   final_model <- lgb.load(nombre_arch)

   prediccion <- predict(
     final_model,
     datos_matrix
  )

  tb_prediccion[, paste0("prob_", isem) := prediccion]
  tb_prediccion[, prob := prob + prediccion]

  rm(final_model)
  rm(prediccion)
  gc(full = TRUE, verbose=FALSE)
}

rm( datos_matrix)
gc(full = TRUE, verbose=FALSE)

tb_prediccion[, prob := prob /length(PARAM$FT$semillas) ]

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2850484,152.3,5544008,296.1,5544008,296.1
Vcells,117403764,895.8,326723535,2492.8,408324268,3115.3


In [ ]:
# veo que hay en tb_prediccion
tb_prediccion

numero_de_cliente,prob,prob_1,prob_2,prob_3,prob_4,prob_5,prob_6,prob_7,prob_8,⋯,prob_31,prob_32,prob_33,prob_34,prob_35,prob_36,prob_37,prob_38,prob_39,prob_40
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
29186441,0.0019950036,0.0016454466,0.0017224270,0.0019149773,0.0020659625,0.0014631397,0.0017481848,0.0017572543,0.0025986655,⋯,0.0021828815,0.0017411670,0.0018860993,0.0021765407,0.0014399611,0.0020944382,0.0025165899,0.0022057822,0.0017925690,0.0024039642
29187961,0.0013009098,0.0012018342,0.0012865262,0.0012906211,0.0011185706,0.0013855724,0.0011448339,0.0014214507,0.0012738265,⋯,0.0012449275,0.0013023475,0.0014291470,0.0017634801,0.0011075690,0.0013905060,0.0013272813,0.0009637876,0.0011388584,0.0015504129
29193101,0.0006314877,0.0006359346,0.0006911280,0.0006918402,0.0007155498,0.0006949136,0.0005734821,0.0004956385,0.0005825438,⋯,0.0005397135,0.0006488222,0.0004237986,0.0007946787,0.0007605805,0.0007734286,0.0005861747,0.0007039239,0.0005663816,0.0005319512
29193281,0.0004181990,0.0005001242,0.0005455361,0.0004198713,0.0004446761,0.0004821942,0.0004632669,0.0003644197,0.0003285684,⋯,0.0005622377,0.0003591432,0.0003542400,0.0004044343,0.0003922431,0.0003929101,0.0003724899,0.0005032581,0.0005020507,0.0004273458
29198891,0.0012317260,0.0017364140,0.0012392825,0.0011398745,0.0011080570,0.0011034218,0.0011017096,0.0010295357,0.0011690785,⋯,0.0011864685,0.0011532986,0.0012603744,0.0015442754,0.0012010563,0.0020833131,0.0013716446,0.0012936138,0.0013279092,0.0013779575
29200651,0.0113573938,0.0078972005,0.0104658329,0.0098342144,0.0094675030,0.0082904857,0.0128163772,0.0097129271,0.0133330157,⋯,0.0118417362,0.0099620899,0.0117743075,0.0151464011,0.0114632806,0.0127068189,0.0132513722,0.0130667231,0.0104401117,0.0084280528
29201701,0.0007389420,0.0007248028,0.0007638568,0.0008450302,0.0006921496,0.0006683844,0.0005926870,0.0008094613,0.0005952517,⋯,0.0008793610,0.0008838622,0.0007674469,0.0008950355,0.0011038196,0.0007140595,0.0006409958,0.0009588358,0.0006914873,0.0005622842
29205441,0.1152176110,0.0846202731,0.1125631501,0.1816954433,0.0450206893,0.1141800302,0.1763569840,0.0877484373,0.1363335640,⋯,0.1038025590,0.1171856902,0.1034039410,0.1281078845,0.0978071822,0.1092598131,0.1330806520,0.1149905616,0.0796202479,0.1003004170
29207901,0.0132518467,0.0094074975,0.0097661445,0.0126339960,0.0126757013,0.0116084027,0.0109770828,0.0159766169,0.0211967809,⋯,0.0118653529,0.0142594750,0.0195624338,0.0195267352,0.0141107979,0.0143398987,0.0176998101,0.0135082336,0.0095364183,0.0066990852


In [ ]:
# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

#### Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle
Competencia Gerencial

In [ ]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "labo-i-2025-virtual-gerencial"
PARAM$kaggle$cortes <- seq(800, 1300, by = 50)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 
Successfully submitted to LaboI 2025 Virtual gerencial 


In [ ]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

Loading required package: yaml



In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Nov 29 01:45:05 PM 2025"